In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import features

folder = f'..\\Traffic, Investigations _ Other\\' 

stations = pd.read_csv(folder + 'police_stations.csv', delimiter=',')
print(len(stations))
stations.head()


22


,DISTRICT_NUMBER,LOCATION,lat,lng,fips
0,7,Bustleton Ave & Bowler St,40.091030,-75.032680,4.210100e+11
1,14,Haines St & Germantown Ave,40.038425,-75.176942,4.210100e+11
2,8,Academy Rd & Red Lion Rd,40.081760,-74.994452,4.210200e+11
3,2,Harbison Ave & Levick St,40.031562,-75.063272,4.210100e+11
4,35,N Broad St & Champlost St,40.044428,-75.143506,4.210100e+11


In [7]:
population = pd.read_csv(folder + 'philadelphia_population_metrics.csv', delimiter=',')
print(len(population))
population.head()

384


,OBJECTID,GEOGRAPHY_NAME,GEOGRAPHY,COUNT_ALL_RACES_ETHNICITIES,COUNT_WHITE_NH,COUNT_BLACK_NH,COUNT_ASIAN_NH,COUNT_HISPANIC,PERCENT_WHITE_NH,PERCENT_BLACK_NH,PERCENT_ASIAN_NH,PERCENT_HISPANIC,Shape__Area,Shape__Length
0,1,42101000100,Census tract,3478,2890,207,173,126,83.093732,5.951696,4.974123,3.622772,1.202257e+06,4528.078584
1,2,42101000200,Census tract,2937,665,284,1855,79,22.642152,9.669731,63.159687,2.689820,6.516548e+05,3688.958965
2,3,42101000300,Census tract,3169,2290,324,328,135,72.262543,10.224045,10.350268,4.260019,9.319655e+05,5040.148986
3,4,42101000401,Census tract,2125,1049,376,519,107,49.364706,17.694118,24.423529,5.035294,3.978075e+05,2733.448537
4,5,42101000402,Census tract,3142,2455,173,356,109,78.134946,5.506047,11.330363,3.469128,5.174244e+05,3547.054900


In [8]:
crimes = pd.read_csv(folder + 'crimes.csv', delimiter=',')
print(len(crimes))
print(crimes.head())
crimes.value_counts('text_general_code')
crimes.value_counts('psa')

1593142
   objectid psa      dispatch_date_time dispatch_date dispatch_time  \
0    218891   2  2022-09-08 19:21:00+00    2022-09-08      15:21:00   
1    218892   2  2022-09-14 21:41:00+00    2022-09-14      17:41:00   
2    218893   2  2022-09-19 19:23:00+00    2022-09-19      15:23:00   
3    218894   2  2022-08-22 13:38:00+00    2022-08-22      09:38:00   
4    218895   2  2022-08-20 14:31:00+00    2022-08-20      10:31:00   

          location_block   text_general_code        lat        lng  \
0  3200 BLOCK N BROAD ST              Thefts  40.002741 -75.153068   
1  3200 BLOCK N BROAD ST              Thefts  40.002741 -75.153068   
2  3200 BLOCK N BROAD ST              Thefts  40.002741 -75.153068   
3        0 BLOCK WOLF ST  Theft from Vehicle  39.918351 -75.145999   
4        0 BLOCK WOLF ST  Theft from Vehicle  39.918351 -75.145999   

           fips  
0  4.210102e+11  
1  4.210102e+11  
2  4.210102e+11  
3  4.210198e+11  
4  4.210198e+11  


psa
2      530495
1      461310
3      423263
4       72535
2.0     35877
1.0     29746
3.0     27473
A        6656
4.0      5167
R           1
S           1
dtype: int64

In [ ]:
investigations = pd.read_csv(folder + 'investigations.csv', delimiter=',')
investigations['year'] = investigations['datetimeoccur'].apply(lambda x: x.split('-')[0])
investigations.dropna(subset=['lat', 'lng'], inplace=True)

print(len(investigations))
print(investigations.columns)
print(investigations.value_counts('individual_arrested'))

In [10]:
stops = pd.read_csv(folder + 'traffic_stops_philadelphia.csv', delimiter=',')
stops['year'] = stops['date'].apply(lambda x: x.split('-')[0])
stops.dropna(subset=['lat', 'lng'], inplace=True)
print(len(stops))
print(stops.columns)
print(stops.value_counts('arrest_made'))



<ipython-input-10-6e2c1ae53bef>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  stops = pd.read_csv(folder + 'traffic_stops_philadelphia.csv', delimiter=',')


1760399
Index(['objectid', 'date', 'time', 'location', 'lat', 'lng', 'district',
       'service_area', 'subject_age', 'subject_race', 'subject_sex', 'type',
       'arrest_made', 'outcome', 'contraband_found', 'frisk_performed',
       'search_conducted', 'search_person', 'search_vehicle', 'raw_race',
       'raw_individual_contraband', 'raw_vehicle_contraband', 'fips', 'year'],
      dtype='object')
arrest_made
False    1669984
True       90415
dtype: int64


In [11]:
print(stops['lat'].min(), stops['lat'].max())
print(stops['lng'].min(), stops['lng'].max())

39.87503164 40.13789
-75.27569444 -74.95770684


In [13]:

coord_mask = features.get_coord_mask(stops)
print(len(coord_mask.drop_duplicates(['mean_lat', 'mean_lng'])))
stops_agg = pd.merge(stops, coord_mask, on=['lat', 'lng'])
stops_agg = stops_agg.groupby(['year', 'mean_lat', 'mean_lng']).agg({'objectid': 'count', 'arrest_made': 'sum', }).reset_index().rename(columns={'objectid': 'stops', 'arrest_made': 'stop_arrests',  })
stops_agg['dist_to_station'] = stops_agg.apply(lambda x: features.find_closest((x['mean_lat'], x['mean_lng']))[0], axis=1)
stops_agg['closest_station'] = stops_agg.apply(lambda x: features.find_closest((x['mean_lat'], x['mean_lng']))[1], axis=1)
stops_agg.head()


452


,year,mean_lat,mean_lng,stops,stop_arrests,dist_to_station,closest_station
0,2014,39.875032,-75.265095,4,0,0.059170,65th St & Woodland Ave
1,2014,39.875032,-75.254495,21,0,0.054417,65th St & Woodland Ave
2,2014,39.875032,-75.233296,207,6,0.050553,65th St & Woodland Ave
3,2014,39.883794,-75.265095,1,0,0.051886,65th St & Woodland Ave
4,2014,39.883794,-75.254495,95,2,0.046393,65th St & Woodland Ave


In [ ]:
investigations_coord_mask = features.get_coord_mask(investigations)
print(len(investigations_coord_mask.drop_duplicates(['mean_lat', 'mean_lng'])))
investigations_agg = pd.merge(investigations, investigations_coord_mask, on=['lat', 'lng'])
investigations_agg = investigations_agg.groupby(['year', 'mean_lat', 'mean_lng']).agg({'objectid': 'count', 'individual_arrested': 'sum', }).reset_index().rename(columns={'objectid': 'investigations', 'individual_arrested': 'investigation_arrests',  })
investigations_agg['dist_to_station'] = investigations_agg.apply(lambda x: features.find_closest((x['mean_lat'], x['mean_lng']))[0], axis=1)
investigations_agg['closest_station'] = investigations_agg.apply(lambda x: features.find_closest((x['mean_lat'], x['mean_lng']))[1], axis=1)
investigations_agg.head()